In [0]:

file_location = "/FileStore/tables/access_logs.txt"
file_type = "csv" 

infer_schema = "false" 
first_row_is_header = "false"
delimiter = " "  

df = spark.read.format(file_type) \
    .option("inferSchema", infer_schema) \
    .option("header", first_row_is_header) \
    .option("sep", delimiter) \
    .load(file_location)

df.show()


+---------------+----------+--------+---------------+
|            _c0|       _c1|     _c2|            _c3|
+---------------+----------+--------+---------------+
| 161.103.53.215|2025-02-06|09:31:36|        bbc.com|
| 215.213.219.84|2025-02-01|21:20:21|    twitter.com|
|  117.152.133.2|2025-02-18|11:15:41|     github.com|
|   1.127.233.58|2025-02-17|07:22:46|        cnn.com|
|181.125.127.119|2025-02-16|21:54:12|     google.com|
|   194.16.65.15|2025-02-08|15:33:01|  wikipedia.org|
| 97.170.188.148|2025-02-09|09:25:16|  wikipedia.org|
| 239.96.185.164|2025-01-29|00:43:01|     amazon.com|
|  131.101.10.63|2025-02-17|15:15:49|     amazon.com|
|    76.45.153.8|2025-02-04|05:09:29|       bing.com|
| 166.242.38.189|2025-02-03|06:33:33|   linkedin.com|
| 220.181.19.201|2025-02-17|21:34:44|     github.com|
|   78.26.170.97|2025-01-31|21:07:52|   facebook.com|
|  16.248.12.159|2025-01-31|08:52:13|        bbc.com|
| 77.171.111.126|2025-02-06|03:59:06|        cnn.com|
| 85.232.169.165|2025-01-25|

In [0]:
# converter para pandas
import pandas as pd

df_pandas = df.toPandas()

# nomear as colunas 
df_pandas.columns = ["IP", "Data", "Hora", "URL"]

df_pandas.head()

,IP,Data,Hora,URL
0,161.103.53.215,2025-02-06,09:31:36,bbc.com
1,215.213.219.84,2025-02-01,21:20:21,twitter.com
2,117.152.133.2,2025-02-18,11:15:41,github.com
3,1.127.233.58,2025-02-17,07:22:46,cnn.com
4,181.125.127.119,2025-02-16,21:54:12,google.com


In [0]:
acessos_google = df_pandas[df_pandas["URL"].str.contains("google", case=False, na=False)]
print(f"Acessos feitos no Google: {acessos_google.shape[0]}")

Acessos feitos no Google: 2484


In [0]:
top_5_urls = df_pandas["URL"].value_counts().head(5)
print(f"As 5 páginas mais visitadas:\n{top_5_urls}")

As 5 páginas mais visitadas:
stackoverflow.com    650
google.com/mail      649
cnn.com              645
amazon.com           644
facebook.com         640
Name: URL, dtype: int64


In [0]:
accesses_by_date = df_pandas.groupby("Data").size()
print(f"Contagem de acessos por data:\n{accesses_by_date}")

Contagem de acessos por data:
Data
2025-01-19     16
2025-01-20    338
2025-01-21    382
2025-01-22    366
2025-01-23    325
2025-01-24    333
2025-01-25    344
2025-01-26    314
2025-01-27    355
2025-01-28    324
2025-01-29    333
2025-01-30    353
2025-01-31    344
2025-02-01    326
2025-02-02    344
2025-02-03    327
2025-02-04    349
2025-02-05    345
2025-02-06    322
2025-02-07    322
2025-02-08    336
2025-02-09    318
2025-02-10    347
2025-02-11    325
2025-02-12    330
2025-02-13    316
2025-02-14    327
2025-02-15    333
2025-02-16    316
2025-02-17    304
2025-02-18    286
dtype: int64


In [0]:
# Função para classificar o IP como 'internal' ou 'external'
def classify_ip(ip):
    if ip.startswith(("10.", "192.168.")):
        return "internal"
    elif ip.startswith("172."):
        # Extrair o segundo octeto do IP
        second_octet = int(ip.split('.')[1])
        if 16 <= second_octet <= 31:  # Verifica se está na faixa 172.16.0.0 - 172.31.255.255
            return "internal"
    return "external"

# Aplicar a função para criar uma nova coluna 'ip_type'
df_pandas['ip_type'] = df_pandas['IP'].apply(classify_ip)

# Contar os acessos por tipo de IP
accesses_by_ip_type = df_pandas['ip_type'].value_counts().reset_index()

# Renomear as colunas para 'ip_type' e 'count'
accesses_by_ip_type.columns = ['ip_type', 'count']

# Exibir o resultado
print(accesses_by_ip_type)

    ip_type  count
0  external   9968
1  internal     32


In [0]:
google_access_count = df.filter(df["_c3"] == "google.com").count()

print(f"Quantidade de acessos ao google.com: {google_access_count}")


Quantidade de acessos a google.com: 601


In [0]:
page_counts = df.groupBy("_c3").count()

top_5_pages = page_counts.orderBy("count", ascending=False).limit(5)

top_5_pages.show()


+-----------------+-----+
|              _c3|count|
+-----------------+-----+
|stackoverflow.com|  650|
|  google.com/mail|  649|
|          cnn.com|  645|
|       amazon.com|  644|
|     facebook.com|  640|
+-----------------+-----+



In [0]:
accesses_by_date = df.groupBy("_c1").count()

accesses_by_date.orderBy("_c1").show(100, False)

+----------+-----+
|_c1       |count|
+----------+-----+
|2025-01-19|16   |
|2025-01-20|338  |
|2025-01-21|382  |
|2025-01-22|366  |
|2025-01-23|325  |
|2025-01-24|333  |
|2025-01-25|344  |
|2025-01-26|314  |
|2025-01-27|355  |
|2025-01-28|324  |
|2025-01-29|333  |
|2025-01-30|353  |
|2025-01-31|344  |
|2025-02-01|326  |
|2025-02-02|344  |
|2025-02-03|327  |
|2025-02-04|349  |
|2025-02-05|345  |
|2025-02-06|322  |
|2025-02-07|322  |
|2025-02-08|336  |
|2025-02-09|318  |
|2025-02-10|347  |
|2025-02-11|325  |
|2025-02-12|330  |
|2025-02-13|316  |
|2025-02-14|327  |
|2025-02-15|333  |
|2025-02-16|316  |
|2025-02-17|304  |
|2025-02-18|286  |
+----------+-----+



In [0]:
from pyspark.sql.functions import when, col

df_with_ip_type = df.withColumn(
    "ip_type", 
    when(col("_c0").like("10.%"), "internal")  
    .when(col("_c0").like("192.168.%"), "internal")  
    .when(col("_c0").like("172.16.%"), "internal")  
    .when(col("_c0").like("172.17.%"), "internal")  
    .when(col("_c0").like("172.18.%"), "internal")  
    .when(col("_c0").like("172.19.%"), "internal")  
    .when(col("_c0").like("172.20.%"), "internal")  
    .when(col("_c0").like("172.21.%"), "internal")  
    .when(col("_c0").like("172.22.%"), "internal")  
    .when(col("_c0").like("172.23.%"), "internal")  
    .when(col("_c0").like("172.24.%"), "internal")  
    .when(col("_c0").like("172.25.%"), "internal")  
    .when(col("_c0").like("172.26.%"), "internal")  
    .when(col("_c0").like("172.27.%"), "internal")  
    .when(col("_c0").like("172.28.%"), "internal")  
    .when(col("_c0").like("172.29.%"), "internal")  
    .when(col("_c0").like("172.30.%"), "internal")  
    .when(col("_c0").like("172.31.%"), "internal")  
    .otherwise("external") 
)

accesses_by_ip_type = df_with_ip_type.groupBy("ip_type").count()

accesses_by_ip_type.show()

+--------+-----+
| ip_type|count|
+--------+-----+
|internal|   32|
|external| 9968|
+--------+-----+

